In [1]:
pip install xgboost lightgbm

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 2.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.7 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [17]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns

from scipy.stats import rankdata
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, HistGradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier, LGBMRegressor 
from xgboost import XGBClassifier, XGBRegressor
# from catboost import CatBoostClassifier, CatBoostRegressor

# import optuna 

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'March-Mania-2023/Evan-Data/MNCAA_train_538.csv'
file_key_2 = 'March-Mania-2023/Evan-Data/MNCAA_test_538.csv'
file_key_3 = 'March-Mania-2023/Evan-Data/WNCAA_train_538.csv'
file_key_4 = 'March-Mania-2023/Evan-Data/WNCAA_test_538.csv'
file_key_5 = 'March-Mania-2023/Evan-Data/SampleSubmission2023.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

bucket_object_4 = bucket.Object(file_key_4)
file_object_4 = bucket_object_4.get()
file_content_stream_4 = file_object_4.get('Body')

bucket_object_5 = bucket.Object(file_key_5)
file_object_5 = bucket_object_5.get()
file_content_stream_5 = file_object_5.get('Body')

## Reading data files
man_train = pd.read_csv(file_content_stream_1)
# man_test = pd.read_csv(file_content_stream_2)
woman_train = pd.read_csv(file_content_stream_3)
# woman_test = pd.read_csv(file_content_stream_4)

submission = pd.read_csv(file_content_stream_5)

man_train['target'] = np.where(man_train['ResultDiff'] > 0, 1, 0)
man_train = man_train[man_train['Season'] >= 2016].reset_index(drop = True)

woman_train['target'] = np.where(woman_train['ResultDiff'] > 0, 1, 0)
woman_train = woman_train[woman_train['Season'] >= 2016].reset_index(drop = True)

man_test = pd.read_csv('man_test_tour_phase_1.csv')
woman_test = pd.read_csv('woman_test_tour_phase_1.csv')

# Man

In [12]:
to_select = ['X1_WinRatio14d',
             'X1_PointsMean',
             'X1_PointsMedian',
             'X1_PointsDiffMean',
             'X1_FgaMean',
             'X1_FgaMedian',
             'X1_FgaMin',
             'X1_FgaMax',
             'X1_AstMean',
             'X1_BlkMean',
             'X1_OppFgaMean',
             'X1_OppFgaMin',
             'X1_EfgpMean',
             'X1_PossessionsMean',
             'X1_PpmMean',
             'X1_FtrMean',
             'X1_TopMean',
             'X1_DrebpMean',
             'X2_WinRatio14d',
             'X2_PointsMean',
             'X2_PointsMedian',
             'X2_PointsDiffMean',
             'X2_FgaMean',
             'X2_FgaMedian',
             'X2_FgaMin',
             'X2_FgaMax',
             'X2_AstMean',
             'X2_BlkMean',
             'X2_OppFgaMean',
             'X2_OppFgaMin',
             'X2_EfgpMean',
             'X2_PossessionsMean',
             'X2_PpmMean',
             'X2_FtrMean',
             'X2_TopMean',
             'X2_DrebpMean',
             'Seed1',
             'Seed2',
             'SeedDiff',
             'quality_march_T1',
             'quality_march_T2', 
             'ResultDiff',
             'X1_team_rating',
             'X1_rd1_win',
             'X1_rd2_win',
             'X1_rd3_win',
             'X1_rd4_win',
             'X1_rd5_win',
             'X1_rd6_win',
             'X1_rd7_win',
             'X2_team_rating',
             'X2_rd1_win',
             'X2_rd2_win',
             'X2_rd3_win',
             'X2_rd4_win',
             'X2_rd5_win',
             'X2_rd6_win',
             'X2_rd7_win']

X = man_train[to_select]
Y = man_train['target']

man_test_tour_ID = man_test['ID']
man_test_tour = man_test[to_select]

#############
## XGBoost ##
#############

xgb_params = pd.read_csv('man_XGB_Phase_2_42_Optuna_Hyperparameters.csv')

xgb_md = XGBClassifier(tree_method = 'hist', 
                       max_depth = xgb_params['max_depth'][0],
                       learning_rate = xgb_params['learning_rate'][0],
                       n_estimators = xgb_params['n_estimators'][0],
                       gamma = xgb_params['gamma'][0],
                       min_child_weight = xgb_params['min_child_weight'][0],
                       colsample_bytree = xgb_params['colsample_bytree'][0],
                       subsample = xgb_params['subsample'][0]).fit(X, Y)

xgb_pred = xgb_md.predict_proba(man_test_tour)[:, 1]

##############
## LightGBM ##
##############

lgb_params = pd.read_csv('man_LightGBM_Phase_2_42_Optuna_Hyperparameters.csv')

lgb_md = LGBMClassifier(boosting_type = 'gbdt', 
                        n_estimators = lgb_params['n_estimators'][0],
                        learning_rate = lgb_params['learning_rate'][0],
                        max_depth = lgb_params['max_depth'][0],
                        lambda_l1 = lgb_params['lambda_l1'][0],
                        lambda_l2 = lgb_params['lambda_l2'][0],
                        num_leaves = lgb_params['num_leaves'][0],
                        bagging_fraction = lgb_params['bagging_fraction'][0],
                        feature_fraction = lgb_params['feature_fraction'][0]).fit(X, Y)

lgb_pred = lgb_md.predict_proba(man_test_tour)[:, 1]

##################
## HistGradient ##
##################

hist_params = pd.read_csv('man_Hist_Phase_2_42_Optuna_Hyperparameters.csv')

hist_md = HistGradientBoostingClassifier(l2_regularization = hist_params['l2_regularization'][0],
                                         early_stopping = False,
                                         learning_rate = hist_params['learning_rate'][0],
                                         max_iter = hist_params['max_iter'][0],
                                         max_depth = hist_params['max_depth'][0],
                                         max_bins = hist_params['max_bins'][0],
                                         min_samples_leaf = hist_params['min_samples_leaf'][0],
                                         max_leaf_nodes = hist_params['max_leaf_nodes'][0]).fit(X, Y)

hist_pred = hist_md.predict_proba(man_test_tour)[:, 1]

##############
## Ensemble ##
##############

ens_pred = (xgb_pred + lgb_pred + hist_pred) / 3
man_pred = pd.DataFrame({'ID': man_test_tour_ID, 'Pred': ens_pred})
man_pred.head()

[LightGBM] [Warning] lambda_l1 is set=1.6568558853289863, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.6568558853289863
[LightGBM] [Warning] feature_fraction is set=0.705433568927063, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.705433568927063
[LightGBM] [Warning] lambda_l2 is set=5.241653664300623, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5.241653664300623
[LightGBM] [Warning] bagging_fraction is set=0.8504282163263281, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8504282163263281


,ID,Pred
0,2023_1104_1112,0.948642
1,2023_1104_1113,0.950995
2,2023_1104_1116,0.950323
3,2023_1104_1120,0.950384
4,2023_1104_1124,0.949934


# Woman

In [13]:
to_select = ['X1_WinRatio14d',
             'X1_PointsMean',
             'X1_PointsMedian',
             'X1_PointsDiffMean',
             'X1_FgaMean',
             'X1_FgaMedian',
             'X1_FgaMin',
             'X1_FgaMax',
             'X1_AstMean',
             'X1_BlkMean',
             'X1_OppFgaMean',
             'X1_OppFgaMin',
             'X1_EfgpMean',
             'X1_PossessionsMean',
             'X1_PpmMean',
             'X1_FtrMean',
             'X1_TopMean',
             'X1_DrebpMean',
             'X2_WinRatio14d',
             'X2_PointsMean',
             'X2_PointsMedian',
             'X2_PointsDiffMean',
             'X2_FgaMean',
             'X2_FgaMedian',
             'X2_FgaMin',
             'X2_FgaMax',
             'X2_AstMean',
             'X2_BlkMean',
             'X2_OppFgaMean',
             'X2_OppFgaMin',
             'X2_EfgpMean',
             'X2_PossessionsMean',
             'X2_PpmMean',
             'X2_FtrMean',
             'X2_TopMean',
             'X2_DrebpMean',
             'Seed1',
             'Seed2',
             'SeedDiff',
             'quality_march_T1',
             'quality_march_T2', 
             'ResultDiff',
             'X1_team_rating',
             'X1_rd1_win',
             'X1_rd2_win',
             'X1_rd3_win',
             'X1_rd4_win',
             'X1_rd5_win',
             'X1_rd6_win',
             'X1_rd7_win',
             'X2_team_rating',
             'X2_rd1_win',
             'X2_rd2_win',
             'X2_rd3_win',
             'X2_rd4_win',
             'X2_rd5_win',
             'X2_rd6_win',
             'X2_rd7_win']

X = woman_train[to_select]
Y = woman_train['target']

woman_test_tour_ID = woman_test['ID']
woman_test_tour = woman_test[to_select]

#############
## XGBoost ##
#############

xgb_params = pd.read_csv('woman_XGB_Phase_2_42_Optuna_Hyperparameters.csv')

xgb_md = XGBClassifier(tree_method = 'hist', 
                       max_depth = xgb_params['max_depth'][0],
                       learning_rate = xgb_params['learning_rate'][0],
                       n_estimators = xgb_params['n_estimators'][0],
                       gamma = xgb_params['gamma'][0],
                       min_child_weight = xgb_params['min_child_weight'][0],
                       colsample_bytree = xgb_params['colsample_bytree'][0],
                       subsample = xgb_params['subsample'][0]).fit(X, Y)

xgb_pred = xgb_md.predict_proba(woman_test_tour)[:, 1]

##############
## LightGBM ##
##############

lgb_params = pd.read_csv('woman_LightGBM_Phase_2_42_Optuna_Hyperparameters.csv')

lgb_md = LGBMClassifier(boosting_type = 'gbdt', 
                        n_estimators = lgb_params['n_estimators'][0],
                        learning_rate = lgb_params['learning_rate'][0],
                        max_depth = lgb_params['max_depth'][0],
                        lambda_l1 = lgb_params['lambda_l1'][0],
                        lambda_l2 = lgb_params['lambda_l2'][0],
                        num_leaves = lgb_params['num_leaves'][0],
                        bagging_fraction = lgb_params['bagging_fraction'][0],
                        feature_fraction = lgb_params['feature_fraction'][0]).fit(X, Y)

lgb_pred = lgb_md.predict_proba(woman_test_tour)[:, 1]

##################
## HistGradient ##
##################

hist_params = pd.read_csv('woman_Hist_Phase_2_42_Optuna_Hyperparameters.csv')

hist_md = HistGradientBoostingClassifier(l2_regularization = hist_params['l2_regularization'][0],
                                         early_stopping = False,
                                         learning_rate = hist_params['learning_rate'][0],
                                         max_iter = hist_params['max_iter'][0],
                                         max_depth = hist_params['max_depth'][0],
                                         max_bins = hist_params['max_bins'][0],
                                         min_samples_leaf = hist_params['min_samples_leaf'][0],
                                         max_leaf_nodes = hist_params['max_leaf_nodes'][0]).fit(X, Y)

hist_pred = hist_md.predict_proba(woman_test_tour)[:, 1]

##############
## Ensemble ##
##############

ens_pred = (xgb_pred + lgb_pred + hist_pred) / 3
woman_pred = pd.DataFrame({'ID': woman_test_tour_ID, 'Pred': ens_pred})
woman_pred.head()

[LightGBM] [Warning] lambda_l1 is set=0.0474587633751369, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0474587633751369
[LightGBM] [Warning] feature_fraction is set=0.6539801342710443, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6539801342710443
[LightGBM] [Warning] lambda_l2 is set=0.3422853948073336, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.3422853948073336
[LightGBM] [Warning] bagging_fraction is set=0.8976619661127543, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8976619661127543


,ID,Pred
0,2023_1104_1112,0.898688
1,2023_1104_1113,0.928590
2,2023_1104_1116,0.921148
3,2023_1104_1120,0.921199
4,2023_1104_1124,0.907645


In [14]:
submission.head()

,ID,Pred
0,2023_1101_1102,0.5
1,2023_1101_1103,0.5
2,2023_1101_1104,0.5
3,2023_1101_1105,0.5
4,2023_1101_1106,0.5


In [18]:
submission = pd.merge(submission, man_pred, on = 'ID', how = 'left')
submission = pd.merge(submission, woman_pred, on = 'ID', how = 'left')
submission.head(20)

,ID,Pred_x,Pred_y,Pred
0,2023_1101_1102,0.5,NaN,NaN
1,2023_1101_1103,0.5,NaN,NaN
2,2023_1101_1104,0.5,NaN,NaN
3,2023_1101_1105,0.5,NaN,NaN
4,2023_1101_1106,0.5,NaN,NaN
5,2023_1101_1107,0.5,NaN,NaN
6,2023_1101_1108,0.5,NaN,NaN
7,2023_1101_1110,0.5,NaN,NaN
8,2023_1101_1111,0.5,NaN,NaN
9,2023_1101_1112,0.5,NaN,NaN


In [19]:
submission.columns = ['ID', 'pred1', 'pred2', 'pred3']
submission.head()

,ID,pred1,pred2,pred3
0,2023_1101_1102,0.5,NaN,NaN
1,2023_1101_1103,0.5,NaN,NaN
2,2023_1101_1104,0.5,NaN,NaN
3,2023_1101_1105,0.5,NaN,NaN
4,2023_1101_1106,0.5,NaN,NaN


In [20]:
submission['Pred'] = np.where(np.isnan(submission['pred2']), submission['pred1'], submission['pred2'])
submission.head()

,ID,pred1,pred2,pred3,Pred
0,2023_1101_1102,0.5,NaN,NaN,0.5
1,2023_1101_1103,0.5,NaN,NaN,0.5
2,2023_1101_1104,0.5,NaN,NaN,0.5
3,2023_1101_1105,0.5,NaN,NaN,0.5
4,2023_1101_1106,0.5,NaN,NaN,0.5


In [21]:
submission['Pred'] = np.where(np.isnan(submission['pred3']), submission['Pred'], submission['pred3'])
submission.drop(columns = ['pred1', 'pred2', 'pred3'], axis = 1, inplace = True)
submission.head()

,ID,Pred
0,2023_1101_1102,0.5
1,2023_1101_1103,0.5
2,2023_1101_1104,0.5
3,2023_1101_1105,0.5
4,2023_1101_1106,0.5


In [23]:
submission.describe()

,Pred
count,130683.000000
mean,0.500113
std,0.054271
min,0.066861
25%,0.500000
50%,0.500000
75%,0.500000
max,0.932993


In [24]:
submission.to_csv('First_submission.csv', index = False)